In [ ]:
import pandas as pd
import json

fda_data = r'drug-ndc-0001-of-0001.json\drug-ndc-0001-of-0001.json'

smiles_from_clin_tri = r"resolved_smiles.txt"

# used to find extra smiles data based on fda own unii identifier
unii_records = r'UNII_Data_20240622\UNII_Records_22Jun2024.txt'



with open(fda_data, 'r', encoding='utf-8') as file:
    json_data = json.load(file)

results = json_data.get('results', [])

fda_drugs = pd.json_normalize(results)


clinical_trials = pd.read_csv(smiles_from_clin_tri, sep = "\t", header = None)
clinical_trials.rename(columns = {0: "name", 1: "SMILES"}, inplace = True)


unii_lookup_df = pd.read_csv(unii_records, delimiter='\t')




In [ ]:
fda_drugs['FDA_Approval'] = 0

# same as removing yellow and red highlighted marketing status from the project
fda_drugs.loc[fda_drugs['application_number'].notnull(), 'FDA_Approval'] = 1


In [ ]:

fda_drugs['original_drug_id'] = fda_drugs.index

# splits multiple drugs into separate rows
df_single_drugs = fda_drugs.explode('openfda.unii')

In [ ]:

sparse_single_drug_data = df_single_drugs[['marketing_start_date', "marketing_category", "original_drug_id", 'FDA_Approval', 'openfda.unii']]

sparse_single_drug_data.rename(columns={'openfda.unii': 'unii'}, inplace=True)

In [ ]:


unii_lookup_df.rename(columns={'UNII': 'unii'}, inplace=True)


unii_lookup_df = unii_lookup_df[['unii', 'SMILES']]


fda_extended = pd.merge(sparse_single_drug_data, unii_lookup_df, on='unii', how='left')

fda_extended

fda_extended["unii"].value_counts()

In [ ]:
fda_extended["unii"].value_counts()

In [ ]:
# only smiles of length 3 or more are valid, couldnt be converted later to graph
fda_extended['smiles_length'] = fda_extended['SMILES'].str.len()

fda_extended = fda_extended[fda_extended['smiles_length'] >= 3]

fda_extended = fda_extended.drop(columns=['smiles_length'])

fda_extended = fda_extended.dropna(subset=['SMILES'])

fda_extended.sort_values(by=['marketing_start_date'], inplace=True)

# removes duplicates based on marketing start date and unii, first one is kept based on time
fda_extended.drop_duplicates(subset=['unii'], keep="first", inplace=True)




In [59]:
# if marketing_category is BLA drop row
fda_extended = fda_extended[fda_extended['marketing_category'] != 'BLA']

# drop marketing_category column
fda_extended = fda_extended.drop(columns=['marketing_category'])

In [ ]:
# final merge with clinical trials data

new_rows = []
for i in clinical_trials["SMILES"]:
    if i not in fda_extended["SMILES"].values:
        new_rows.append({"SMILES": i, "FDA_Approval": 0})

if new_rows:
    final_df = pd.concat([fda_extended, pd.DataFrame(new_rows)], ignore_index=True)

In [ ]:
final_df.to_csv('Finished_data.csv', index=False)